In [4]:
import psycopg2
import pandas as pd

# Establish connection to Database

In [5]:
conn = psycopg2.connect(
    dbname="mypostgres", 
    user="postgres", 
    password="YourPassword", 
    host="localhost", 
    port="5432"
)

In [6]:
cur = conn.cursor()

# Read in the excel files

In [7]:
df_green_taxi = pd.read_csv('green_tripdata_2019-09.csv')
df_taxi_zone = pd.read_csv('taxi+_zone_lookup.csv')

/tmp/ipykernel_3436/2647400118.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_green_taxi = pd.read_csv('green_tripdata_2019-09.csv')


In [8]:
pip install sqlalchemy


Note: you may need to restart the kernel to use updated packages.


In [9]:
from sqlalchemy import create_engine

# Replace the following with your actual database credentials
user = 'postgres'
password = 'YourPassword'
host = 'localhost'
port = '5432'
db = 'mypostgres'

engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{db}')


# Bring in the files into postgres DB table

In [10]:
df_green_taxi.to_sql('green_taxi_trips', engine, if_exists='replace', index=False)
df_taxi_zone.to_sql('taxi_zone_lookup', engine, if_exists='replace', index=False)


265

# Question 3. Count records

In [11]:
with engine.connect() as connection:
    result = connection.execute("""
        SELECT COUNT(*)
        FROM green_taxi_trips
        WHERE 
            DATE(lpep_pickup_datetime) = '2019-09-18'
            AND DATE(lpep_dropoff_datetime) = '2019-09-18';
    """)
    count = result.scalar()
    print(count)


15612


# Question 4. Largest trip for each day

In [13]:
with engine.connect() as connection:
    result = connection.execute("""
        SELECT DATE(lpep_pickup_datetime) AS pickup_day, SUM(trip_distance) AS total_distance
        FROM green_taxi_trips
        GROUP BY pickup_day
        ORDER BY total_distance DESC
        LIMIT 1;
    """)
    for row in result:
        print(row['pickup_day'])

2019-09-26


# Question 5. Three biggest pick up Boroughs

In [14]:
with engine.connect() as connection:
    result = connection.execute("""
        SELECT z."Borough", SUM(t.total_amount) AS total_amount_sum
        FROM green_taxi_trips t
        JOIN taxi_zone_lookup z ON t."PULocationID" = z."LocationID"
        WHERE DATE(t.lpep_pickup_datetime) = '2019-09-18' 
            AND z."Borough" <> 'Unknown'
        GROUP BY z."Borough"
        HAVING SUM(t.total_amount) > 50000
        ORDER BY total_amount_sum DESC
        LIMIT 3
    """)
    for row in result:
        print(row['Borough'], row['total_amount_sum'])

Brooklyn 96333.24000000037
Manhattan 92271.2999999995
Queens 78671.71000000002


# Question 6. Largest tip

In [15]:
with engine.connect() as connection:
    result = connection.execute("""
        SELECT z_dropoff."Zone" AS dropoff_zone, MAX(t.tip_amount) AS max_tip
        FROM green_taxi_trips t
        JOIN taxi_zone_lookup z_pickup ON t."PULocationID" = z_pickup."LocationID"
        JOIN taxi_zone_lookup z_dropoff ON t."DOLocationID" = z_dropoff."LocationID"
        WHERE DATE(t.lpep_pickup_datetime) >= '2019-09-01' 
            AND DATE(t.lpep_pickup_datetime) < '2019-10-01'
            AND z_pickup."Zone" = 'Astoria'
        GROUP BY z_dropoff."Zone"
        ORDER BY max_tip DESC
        LIMIT 1;
    """)
    first_row = result.fetchone()
    print(first_row)

('JFK Airport', 62.31)
